In [16]:
import pandas as pd 
import numpy as np 
import re

from time import sleep
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException

In [52]:
import glob 
done = glob.glob("stocks/*.csv")
done = [item[7: item.find(".csv")] for item in done]
print(done)

['AAPL', 'ABNB', 'ACGL', 'ACQRW', 'ADBE', 'ADI', 'ADP', 'ADSK', 'AEP', 'AGNCP', 'AHPAW', 'AKAM', 'ALCO', 'ALGN', 'ALNY', 'AMAT', 'AMD', 'AMGN', 'AMZN', 'ANSS', 'ANTE', 'APP', 'ASML', 'ATVI', 'AUUDW', 'AVGO', 'AZN', 'BANFP', 'BBQ', 'BGNE', 'BIDU', 'BIIB', 'BJDX', 'BKNG', 'BKR', 'BLEUR', 'BMRN', 'BNGO', 'BNTX', 'CATC', 'CCEP', 'CDNS', 'CDW', 'CEG', 'CERN', 'CFIVU', 'CG', 'CHI', 'CHKP', 'CHTR', 'CINF', 'CMCAU', 'CMCSA', 'CME', 'CNET', 'COIN', 'COST', 'CPRT', 'CRNC', 'CRWD', 'CSCO', 'CSGP', 'CSIQ', 'CSX', 'CTAS', 'CTSH', 'CYTHW', 'CYTO', 'CZR', 'DCOMP', 'DCRDU', 'DDOG', 'DISCA', 'DISCB', 'DISCK', 'DISH', 'DLTR', 'DOCU', 'DXCM', 'EA', 'EBAY', 'ELEV', 'ENPH', 'ENTG', 'EOSE', 'EQIX', 'ERIC', 'ESQ', 'ETSY', 'EXC', 'EXPD', 'EXPE', 'FANG', 'FAST', 'FB', 'FISV', 'FITB', 'FLNT', 'FOX', 'FOXA', 'FTNT', 'FWONK', 'GET', 'GFS', 'GILD', 'GMAB', 'GNAC', 'GOEV', 'GOOG', 'GOOGL', 'GRAB', 'HBAN', 'HGEN', 'HOLX', 'HON', 'HPK', 'HZNP', 'ICLR', 'IDXX', 'IEP', 'IGIC', 'ILMN', 'IMKTA', 'INCY', 'INTC', 'INTU', '

In [2]:
stocks = pd.read_csv("nasdaq.csv")
symbols = list(stocks.sort_values(by = "Market Cap", ascending = False)["Symbol"][:200])

In [4]:
tick = 'msft'
link = f"https://www.morningstar.com/stocks/xnas/{tick}/sustainability"
print(link)

https://www.morningstar.com/stocks/xnas/msft/sustainability


In [7]:
df = pd.DataFrame({"symbols": symbols})
df["link"] = df.apply(lambda row: f"https://www.morningstar.com/stocks/xnas/{row.symbols}/sustainability", axis = 1)

In [19]:
ser = Service("D:/ugahacks7/api/yahoo_scraper/chromedriver_win32/chromedriver.exe")
driver = webdriver.Chrome(service = ser)
driver.get(link)

In [23]:
for item in driver.find_elements(By.CLASS_NAME, "issues-content"): 
    print(item.text)

Data Privacy & Security
Overall
Human Capital
Overall
Business Ethics
Overall


In [42]:
rating = driver.find_element(By.CLASS_NAME, "sal-eqsv-sustainability-rating")
score = len(rating.find_element(By.CLASS_NAME, "rating-content").find_elements(By.CLASS_NAME, "rating"))
print(score)

4


In [47]:
risk = driver.find_element(By.CLASS_NAME, "sal-si__flag-text").text
print(risk)

13.26
Low


In [48]:
driver.find_element(By.CLASS_NAME, "incidents").text

'Incidents: Customer'

In [63]:
df["data"] = ""
for i, row in df.iterrows(): 
    driver.get(row.link)
    sleep(2)
    issues = []
    for item in driver.find_elements(By.CLASS_NAME, "issues-content"): 
        if item.text != 'Overall':
            issues.append(item.text)
        
    rating = driver.find_element(By.CLASS_NAME, "sal-eqsv-sustainability-rating")
    score = len(rating.find_element(By.CLASS_NAME, "rating-content").find_elements(By.CLASS_NAME, "rating"))
    
    incidents = driver.find_element(By.CLASS_NAME, "incidents").text
    df.loc[i, "data"] = (issues, score, incidents)
    
    if i == 10: 
        break

,symbols,link,data
0,AAPL,https://www.morningstar.com/stocks/xnas/AAPL/s...,([Business Ethics\nOverall\nHuman Capital\nOve...
1,MSFT,https://www.morningstar.com/stocks/xnas/MSFT/s...,([Data Privacy & Security\nOverall\nHuman Capi...
2,GOOG,https://www.morningstar.com/stocks/xnas/GOOG/s...,([Business Ethics\nOverall\nHuman Capital\nOve...
3,GOOGL,https://www.morningstar.com/stocks/xnas/GOOGL/...,([Business Ethics\nOverall\nHuman Capital\nOve...
4,AMZN,https://www.morningstar.com/stocks/xnas/AMZN/s...,([Business Ethics\nOverall\nData Privacy & Sec...
5,TSLA,https://www.morningstar.com/stocks/xnas/TSLA/s...,([Product Governance\nOverall\nHuman Capital\n...
6,NVDA,https://www.morningstar.com/stocks/xnas/NVDA/s...,([Business Ethics\nOverall\nHuman Capital\nOve...
7,FB,https://www.morningstar.com/stocks/xnas/FB/sus...,([Data Privacy & Security\nOverall\nProduct Go...
8,NTES,https://www.morningstar.com/stocks/xnas/NTES/s...,([Human Capital\nOverall\nData Privacy & Secur...
9,ASML,https://www.morningstar.com/stocks/xnas/ASML/s...,([Business Ethics\nOverall\nHuman Capital\nOve...
